In [1]:
from os.path import join, exists, dirname
from glob import glob
import pickle
import mplh.cluster_help as ch
import os
import vireoSNP
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
print(vireoSNP.__version__)

import pandas as pd
import seaborn as sns
from vireoSNP import Vireo
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

from src.external.pyvenn import venn

%matplotlib inline
%load_ext autoreload
%autoreload 2
import src.utils.variant_utils as vu
from src.utils.data_io import wrap_load_mtx_df
from src.utils.variant_utils import get_high, get_low
from icecream import ic
import src.utils.variant_utils as vu
from src.utils.data_io import wrap_load_mtx_df

0.5.6


In [2]:
ic.disable()

In [3]:
from src.lineage.lineage_enrichment import calc_enrich,norm_clone_sizes

In [4]:
#enrich_f = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simple/vireo/nclones20/enrichment/enrichmentNorm_donor0.csv"
#cells_meta_f = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simple/vireo/nclones20/cells_meta.tsv"
#OUTDIR = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simple/vireo/nclones20/enrichment/enrich_qc/"
#samples = "Control,Flt3l"
#names = "Control,Flt3l"
#donor = 0

enrich_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/enrichment/enrichmentNorm.csv"
cells_meta_f  = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/concat/cells_meta.tsv"
donor = 0 
samples= "preB,postB,preA,postA"
OUTDIR = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/enrichment/shuffle_stats/"

# col = "lineage"
# filt_col = "donor"

comparisons = "B,preB,postB;A,preA,postA" # "None"
# Params
# vars_to_plot=10
# sample_names="Control,Flt3l"
# N_DONORS=4



In [5]:
def shuffle_single_clones(df, names):
    df = df.dropna(subset=["lineage"]).copy()
#     print(len(df.sample(frac=1)["lineage"].values))
#     print(df.shape)
    df.loc[:,"lineage"] = df.sample(frac=1)["lineage"].values
    enrich_df, clust_counts = calc_enrich(df, samples=names, verbose=False)
    enrich_stats = norm_clone_sizes(enrich_df, clust_counts=clust_counts, names=names)
    return enrich_stats

# def shuffle_clones(n_sim, cells_df, names):
#     all_enrich = []
#     all_enrich_max = []
#     all_fold = []
#     all_fold_max = []
#     for n in range(n_sim):
#         print('n_sim', n)
#         curr = shuffle_single_clones(cells_df, names)
#         all_enrich.append(curr["Fisher -log10p"].values)
#         all_enrich_max.append(max(curr["Fisher -log10p"].values))
#         all_fold_max.append(max(abs(curr[f'{names[1]} fold enrichment norm'])))
#         all_fold.append(curr[f'{names[1]} fold enrichment norm'])
#     return all_enrich, all_enrich_max, all_fold, all_fold_max

#shuffle_single_clones(cells_meta_df)



In [6]:
from numpanpar import parallel_df as pardf
def shuffle_clones_df(sim_df, cells_df, names):
    all_enrich = []
    all_enrich_max = []
    all_fold = []
    all_fold_max = []
    for ind, val in sim_df.iterrows():
        curr = shuffle_single_clones(cells_df, names)
        all_enrich.append(curr["Fisher -log10p"].values)
        all_enrich_max.append(max(curr["Fisher -log10p"].values))
        all_fold_max.append(max(abs(curr[f'{names[1]} fold enrichment norm'])))
        all_fold.append(curr[f'{names[1]} fold enrichment norm'].values)

    sim_df.loc[:,'fold max'] = all_fold_max
    sim_df.loc[:,'enrich max'] = all_enrich_max
    sim_df.loc[:,'enrich'] = all_enrich #[all_enrich]*len(sim_df)
    sim_df.loc[:,'fold'] = all_fold #[all_fold]*len(sim_df)
    return sim_df


In [7]:
def shuffle_clones(enrich_f, cells_meta_f, outdir, names, name=""):
    enrich_df = pd.read_csv(enrich_f)
    cells_meta_df = pd.read_csv(cells_meta_f, sep='\t')
    to_par = True
    if to_par:
        ### PARALLEL ###
        out = pardf(pd.DataFrame(range(1000)), shuffle_clones_df, func_args=(cells_meta_df, names), num_processes=32)
        all_enrich, all_enrich_max, all_fold, all_fold_max = out['enrich'], out['enrich max'], out['fold'], out['fold max']
    else:
        ### NOT PARALLEL ###
        all_enrich, all_enrich_max, all_fold, all_fold_max = shuffle_clones(n_sim=1000, 
                                                                            cells_meta_df=cells_meta_df,
                                                                            names=names)
    all_enrich_l = []
    for x in all_enrich.values:
        all_enrich_l += list(x)
    

    f, ax = plt.subplots(ncols=2, dpi=300, figsize=(10,10))
    ax[0].hist(np.array(all_enrich_max),  color='r')
    perc_thresh = 95
    perc_val = np.percentile(all_enrich_max, perc_thresh)
    ax[0].vlines(x=perc_val, ymin=0, ymax=ax[0].get_ylim()[1], color='g')

    ax[0].hist((enrich_df['Fisher -log10p']),  color='b')
    #plt.xlim([0,1])
    plt.title(f"Simulated (red) vs Real (blue) enrichment p-value all max clones\n Green is {perc_thresh}% cutoff ")
    perc_thresh = 95
    perc_val = np.percentile(all_enrich_max, perc_thresh)


    for c, c_df in enrich_df.iterrows():
        ax[1].vlines(x=np.clip((enrich_df['Fisher -log10p']), 0, 5), ymin=0, ymax=1
                  , color='b')
        #plt.xlim([0,1])
    ax[1].vlines(x=perc_val, ymin=0, ymax=1, color='r',
             linewidth=3)
    plt.suptitle(f"{name} Clones p-value vs Simulated p-value cutoff")

    plt.savefig(join(outdir, f"{name}donor{donor}.clone_shuffle.pdf"))
    
    enrich_df["Comparison"] = name
    enrich_df.to_csv(join(outdir, f"{name}donor{donor}.clone_shuffle.csv"))
    return enrich_df

In [9]:
print('comparisons', comparisons)
if comparisons is not None and comparisons != "None":
    comps = comparisons.split(";")
    print('comps', comps)
    comps_out = []
    comps_out_csv = []
    for c in comps:
        name, a, b = c.split(',')
        print('name, a,b', name, a, b)
        samples = [a, b]
        enrich_d = dirname(enrich_f)
        curr_enrich_df = shuffle_clones(join(enrich_d,f"{name}_enrichmentNorm.csv"), cells_meta_f, outdir=OUTDIR,
                                   names=[a, b], name=f"{name}_")
        comps_out.append(join(OUTDIR, f"{name}_donor{donor}.clone_shuffle.pdf"))
        comps_out_csv.append(curr_enrich_df)
    cmd = f"convert -append {' '.join(comps_out)} {join(OUTDIR, f'donor{donor}.clone_shuffle.pdf')}"
    print('cmd', cmd)
    os.system(cmd)
    df = pd.concat(comps_out_csv, axis=0)
    print('df', df.head())
    df.to_csv(join(OUTDIR, f"donor{donor}.clone_shuffle.csv"))
else:
    samples = samples.split(',')
    shuffle_clones(enrich_f, cells_meta_f, outdir=OUTDIR, names=samples)

comparisons B,preB,postB;A,preA,postA
comps ['B,preB,postB', 'A,preA,postA']
name, a,b B preB postB
ERROR! Session/line number was not unique in database. History logging moved to new session 37
name, a,b A preA postA
cmd convert -append /data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/enrichment/shuffle_stats/B_donor0.clone_shuffle.pdf /data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/enrichment/shuffle_stats/A_donor0.clone_shuffle.pdf /data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/e